In [7]:
import pandas as pd

# โหลดข้อมูลจาก CSV
data1 = pd.read_csv("data_with_cluster.csv")
data1.shape

(46593, 9)

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib

# เตรียมข้อมูล
le = LabelEncoder()
data1['cluster'] = le.fit_transform(data1['cluster'])
X = data1['title']
y = data1['cluster']

# แบ่งข้อมูล
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้าง Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', XGBClassifier(
        tree_method='gpu_hist',      # ใช้ GPU สำหรับการ train
        predictor='gpu_predictor',  # ใช้ GPU สำหรับการ predict
        n_jobs=-1                   # ใช้ทุก core ที่มี
    ))
])

# กำหนดพารามิเตอร์สำหรับ GridSearchCV
param_grid = {
    'tfidf__max_df': [0.8, 0.9, 1.0],
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [3, 6, 10],
    'clf__learning_rate': [0.01, 0.1, 0.3],
    'clf__subsample': [0.8, 1.0]
}

# ใช้ GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,              # จำนวน folds
    n_jobs=-1,         # ใช้ทุก core
    verbose=3          # แสดง log การรัน
)

# ฝึกโมเดล
grid_search.fit(X_train, y_train)

# แสดงพารามิเตอร์ที่ดีที่สุด
print("Best Parameters:", grid_search.best_params_)

# บันทึกโมเดล
joblib.dump(grid_search.best_estimator_, 'model/model_with_gpu.joblib')

# ทดสอบ accuracy
y_pred = grid_search.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib

# เตรียมข้อมูล
le = LabelEncoder()
data1['Cluster'] = le.fit_transform(data1['Cluster'])
X = data1['title']
y = data1['Cluster']

# แบ่งข้อมูลเป็น train และ test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้าง Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_df=0.8)),
    ('clf', XGBClassifier(
        tree_method='gpu_hist',
        learning_rate=0.1,
        max_depth=10,
        n_estimators=200,
        subsample=0.8,
        predictor='gpu_predictor',  # ใช้ GPU สำหรับการ predict
        n_jobs=-1                 # ใช้ทุก core ที่มี
    ))
])

# ฝึกโมเดล
pipeline.fit(X_train, y_train)

# ทดสอบ accuracy
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# บันทึกโมเดลที่ฝึกเสร็จแล้ว
joblib.dump(pipeline, 'model/model_with_gpu.joblib')


c:\Users\PREDATOR\anaconda3\envs\dsde-cedt\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:46:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\PREDATOR\anaconda3\envs\dsde-cedt\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:46:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\PREDATOR\anaconda3\envs\dsde-cedt\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:48:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost

Accuracy: 0.22309260650284365


['model/model_with_gpu.joblib']

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib
import string
import nltk
from nltk.corpus import stopwords  # เพิ่มการ import stopwords

# โหลด stopwords จาก nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# ฟังก์ชั่นสำหรับการตัดคำ stopwords
def remove_stopwords(text):
    # ตัดเครื่องหมายวรรคตอน
    text = ''.join([char for char in text if char not in string.punctuation])
    # ตัดคำ stopwords
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# เตรียมข้อมูล (ตัวอย่าง)
import pandas as pd


# เตรียมข้อมูล
data1['title'] = data1['title'].apply(remove_stopwords)

# การทำ Label Encoding สำหรับ Cluster
le = LabelEncoder()
data1['Cluster'] = le.fit_transform(data1['Cluster'])  # label encoding สำหรับ Cluster

# ข้อมูล Features และ Labels
X = data1['title']  # ข้อความ (title)
y = data1['Cluster']  # คลัสเตอร์ที่เกิดจาก DBSCAN

# แบ่งข้อมูลเป็น training set และ test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้าง pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_df=0.8)),  # แปลงข้อความเป็นเวกเตอร์
    ('clf', SVC(kernel='linear', C=1, gamma='scale', probability=True))  # ใช้ SVM กับ kernel แบบ linear
])

# ฝึกโมเดล
pipeline.fit(X_train, y_train)

# ทดสอบ accuracy
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# บันทึกโมเดล
joblib.dump(pipeline, 'model/model_with_stopwords_removed.joblib')

# ใช้โมเดลในการทำนาย cluster จากข้อความใหม่
def predict_cluster(title):
    model = joblib.load('model/model_with_stopwords_removed.joblib')
    prediction = model.predict([title])  # ทำนาย cluster จาก title
    return le.inverse_transform(prediction)  # แปลงผลลัพธ์กลับเป็น label เดิม

# ตัวอย่างการใช้ฟังก์ชั่น
title_input = "Enter your keyword or title here"
predicted_cluster = predict_cluster(title_input)
print(f"The title belongs to cluster: {predicted_cluster[0]}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PREDATOR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.5029509604034768
The title belongs to cluster: 0


In [14]:
title_input = "Resource reconfiguration by surviving SMEs in a disrupted industry"
predicted_cluster = predict_cluster(title_input)
print(f"The title belongs to cluster: {predicted_cluster[0]}")

The title belongs to cluster: 5
